# 그 밖의 영상 처리 문제

다중 레이블 분류는 이미지 내에 포함된 하나 이상의 물체 범주를 식별하는 문제이다. 물체가 여러 종류 존재할 수도 있지만, 찾고 싶은 종류의 물체가 전혀 없는 상황도 모두 포함된다.

In [1]:

from fastai.vision.all import *

In [2]:
path = untar_data(URLs.PASCAL_2007)

In [3]:
df = pd.read_csv(path / 'train.csv')
df.head()

,fname,labels,is_valid
0,000005.jpg,chair,True
1,000007.jpg,car,True
2,000009.jpg,horse person,True
3,000012.jpg,car,False
4,000016.jpg,bicycle,True


## class 설명
* `Dataset` : 단일 데이터를 표현하는 독립변수 및 종속변수 튜플을 반환한다.
* `DataLoader` : 미니배치 스트림을 제공하는 반복자로, 각 미니배치는 여러 독립변수와 여러 종속변수에 대한 튜플로 구성된다.
* `Datasets` : 학습용 및 검증용 `Dataset`을 포함하는 반복자이다.
* `DataLoaders` : 학습용 및 검증용 `DataLoader`를 포함하는 객체이다. 

In [4]:
def get_x(r): return r['fname']


def get_y(r): return r['labels']

In [5]:
def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train, valid

In [6]:
dblock = DataBlock(
    blocks=(ImageBlock, MultiCategoryBlock),
    splitter=splitter,
    get_x=get_x,
    get_y=get_y,
    item_tfms=RandomResizedCrop(128, min_scale=0.35)
)
dls = dblock.dataloaders(df)

FileNotFoundError: [Errno 2] No such file or directory: '000012.jpg'

In [ ]:
dsets = dblock.datasets(df)

In [ ]:
dsets.train[0]

In [ ]:
idxs = torch.where(dsets.train[0][1] == 1.)[0]
dsets.train.vocab[idxs]

In [ ]:
dls.show_batch(nrows=1, ncols = 3)

In [ ]:
learn = visual_learner(dls, resnet18)

In [ ]:
x, y = dls.train.one_batch()
activs = learn.model(x)
activs.shape

In [ ]:
activs[0]

In [ ]:
def binary_cross_entropy(inputs, targets):
    inputs = inputs.sigmoid()
    return -torch.where(targets == 1, 1 - inputs, inputs).log().mean()

In [ ]:
loss_func = nn.BCEWighLogitsLoss()
loss = loss_func(activs, y)
loss

In [ ]:
def accuracy(inp, targ, axis=-1):
    pred = inp.argmax(axis=axis)
    return (pred == targ).float().mean()

In [ ]:
def accuracy_multi(inp, targ, thresh=0.5, sigmoid=True):
    if sigmoid: inp = inp.sigmoid()
    return ((inp>thresh)==targ.bool()).float().mean()